<div class='alert-block alert-info'>
    <br>
    <h1 align="center"><b>  Lab session 3 :</b> Local search </h1>
    <h3 align="center">Artificial Intelligence Algorithms </h3>
    <h5 align="center">MESIIN476024</a></h5>
    <br>
</div>

#### CONTENTS

* **Part I: Traveling Salesman Problem**</br>
    * Problem definition</br>
* **Part II: Greedy search**</br>
* **Part III: Local search algorithms**</br>
    1. Hill Climbing</br>
    2. Simulated Annealing</br>
    3. Genetic Algorithm</br>
    
</br>

#### **Overview**
In this lab session, you will apply your understanding by implementing local search algorithms as discussed in Lectures 4 and 5. We'll explore the Traveling Salesman Problem (TSP) using Romanian capital cities as an example.  The TSP is an optimization problem  that seeks the shortest possible route visiting each city exactly once, with the journey starting and ending in the same city, thereby forming a closed loop. TSP has numerous practical applications in logistics, manufacturing, and computer networking. Solving TSP is crucial for optimizing resource utilization and improving efficiency in various industries.

To gain a comprehensive understanding of these concepts, take the time to thoroughly engage with the detailed descriptions provided in the notebook. Show your understanding by answering the questions and completing the requested implementations.

<h1 align="left"> <font color='darkblue'> <b>Part I: Problem definition - Traveling Salesman Problem </b></font></a></h1>  


In the following, we propose using the Romanian map to address the Traveling Salesman Problem.
To do this, we need to build a suitable representation of the problem domain. The choice of representation can significantly affect the performance of search optimization techniques. Since the TSP involves finding a closed loop that visits each city exactly once, we will represent each city as a tuple containing the city name and its position, specified by an (x, y) location on a grid. The _state_ (_Node_) will therefore consist of an ordered sequence (i.e., a list) of cities, with the path defined as the sequence generated by traveling from each city in the list to the next in order.


For calculating the distances between every pair of cities on the Romanian map, we will use the Manhattan Distance. The Manhattan distance between two points, $(x_1,y_1)$ and $(x_2,y_2)$ is given by $|x_1-x_2| + |y_1-y_2|$.

### **I.a. Define Romania map with manhattan distances**

In [1]:
# ===================================================================
# Import necessary modules and libraries

from search import *
from Complementary_notebook import psource, heatmap, gaussian_kernel, show_map2, show_map3, final_path_colors, display_visual, plot_NQueens

# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines
from collections import defaultdict, deque, Counter

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time

In [2]:
# Define a location of each city

# Dictionary mapping each city to its coordinates (x, y) in a 2D plane
locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

# Define the romania map as an undirected graph
romania_map = UndirectedGraph()

# Populate the graph with connections between all cities
for city1 in locations:
    for city2 in locations:
        if city1 != city2:
            distance = manhattan_distance(locations[city1], locations[city2])
            romania_map.connect(city1, city2, distance)


# Store the locations dictionary to use later in the graph representation
romania_locations =locations


In [ ]:
# Define the visual attributes for the nodes in the graph

# Set all nodes to have the initial color 'white'
node_colors = {node: 'white' for node in locations.keys()}
# Set the positions of nodes based on the coordinates defined earlier
node_positions = locations

# Define the label positions for each node (slightly offset from the actual node position)
# This helps in making the labels visible without overlapping the node marker itself
node_label_pos = {k: [v[0], v[1] - 10] for k, v in locations.items()}

# Define the edge weights for each connection in the graph
# Extract the weights from the graph dictionary created earlier
edge_weights = {(k, k2): v2 for k, v in romania_map.graph_dict.items() for k2, v2 in v.items()}

# Collect all graph-related data into one dictionary to be used for visualization
romania_graph_data = {
    'graph_dict': romania_map.graph_dict,  # Graph dictionary representing connections between cities
    'node_colors': node_colors,            # Colors of the nodes
    'node_positions': node_positions,      # Positions of the nodes on the 2D plane
    'node_label_positions': node_label_pos,  # Positions of labels for the nodes
    'edge_weights': edge_weights           # Weights of the edges (distance between nodes)
}

# Call a function to visualize the Romania map
# The function 'show_map2' takes in the data and creates a graphical representation of the map
show_map2(romania_graph_data)


### **I.b. Define the TSP problem**

We need to create a class for this problem. The "Problem" class from the _search.py_ module will be used as the base class.

In [4]:
# ===================================================================
# defining the TSP problem

class TSP_problem(Problem):

    """ subclass of Problem to define various functions """

    def two_opt(self, state):
        """ Neighbour generating function for Traveling Salesman Problem
        This function generates a neighbour state by reversing part of the path
        (e.g., reverse the BCD sequence in [ABCDE] to get [ADCBE] as the successor)
        """

        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state) - 1)
        right = random.randint(0, len(neighbour_state) - 1)
        if left > right:
            left, right = right, left
        neighbour_state[left: right + 1] = reversed(neighbour_state[left: right + 1])
        return neighbour_state

    def actions(self, state):
        """ action that can be excuted in given state """
        return [self.two_opt]

    def result(self, state, action):
        """  result after applying the given action on the given state """
        return action(state)

    def path_cost(self,c, state1, action, state2):
        """ total distance for the Traveling Salesman to be covered if in state2  """
        cost = 0
        for i in range(len(state2) - 1):
            cost += distances[state2[i]][state2[i + 1]]


        cost += distances[state2[0]][state2[-1]]

        return cost

    def value(self, state):
        """ value of path cost given negative for the given state """
        return self.path_cost(None, None, None, state)

1. **two_opt(state):** The two-opt method is a neighbor-generating function that works by selecting any two non-adjacent edges in the tour and reversing the segment between them to determine if this yields an improvement in the total distance of the tour.

2. **actions(state):** Returns the possible actions that can be performed in a given state. In this TSP model, the only available action is the two_opt method, which tries to find a better tour by reversing a random sub-segment of the route.

3. **result(state, action):** Returns the state that results from executing the given action (in this case, two_opt) on the provided state.


4. **path_cost(c, state1, action, state2):** Calculates and returns the total distance of the tour represented by state2. It sums up the distances between consecutive cities in the tour and includes the distance from the last city back to the first to complete the loop.


5. **value(state):** Returns the path cost for the given state. 


We can create an instance of the _TSP_problem_ class as follows:

In [5]:
import numpy as np

distances = {}
all_cities = []

for city in locations.keys():
    distances[city] = {}
    all_cities.append(city)

all_cities.sort()



# List of cities
cities = list(locations.keys())
num_cities = len(cities)

# distances for connected cities
for city1 in all_cities:

    for city2 in all_cities:
        if city2 in romania_map.graph_dict.get(city1, {}):
            distances[city1][city2] = romania_map.graph_dict[city1][city2]


# construct an instance of the TSP
tsp = TSP_problem(all_cities)

### How hard is the TSP problem?

The TSP is an extremely challenging problem. In computational complexity theory, it is classified as NP-hard, which means there is no known algorithm that can solve all instances of this problem in polynomial time. TSP is considered one of the quintessential examples of a combinatorial optimization problem.

The difficulty of the TSP arises because the number of possible routes grows exponentially as the number of cities increases. Each time a new city is added, the number of potential routes multiplies dramatically. Unlike problems like sorting, which can be efficiently divided into smaller subproblems and solved recursively, the TSP does not lend itself well to such divide-and-conquer approaches due to its complexity.

For an instance with **n** cities, there are roughly **(n-1)! / 2** possible routes to evaluate, assuming we treat symmetric routes (i.e., traveling in both directions) as equivalent. This number becomes extremely large even for a modest number of cities. For example, with just **20 cities**, the number of possible routes is around **60 trillion**. This massive search space makes finding the optimal solution computationally impractical for large instances using brute force. This is why heuristic and approximation algorithms, like Genetic Algorithms and Simulated Annealing, are often employed to find good solutions within a reasonable amount of time, even though they may not always guarantee an optimal solution.

In [ ]:
# Size of the TSP problem for the Romania map
len(all_cities)

In [ ]:
def factorial(n):
    retval = 1
    for i in range(1, n + 1):
        retval = retval * i
    return retval

result = factorial(20-1)/2
print("Number of possible routes for a 20-city TSP problem:", result)

<h1 align="left"> <font color='darkblue'> <b>Part II: Greedy search  </b></font></a></h1>  

How can we solve the problem of finding the optimal route? One simple approach is to always pick the closest city to visit next. This is known as a **“greedy” method**, as it selects the nearest option at each step, aiming for the immediate best choice.

But how effective is it? Let’s observe the following code that implements this approach and examine the computed result.

In [ ]:
def greedy_tsp(problem, start=None):
    """Greedy algorithm to solve the TSP."""
    current_city = start or random.choice(problem.initial)
    path = [current_city] # Initialize the path with the starting city
    unvisited_cities = set(problem.initial) - {current_city}  # Set of cities yet to be visited

    # Visit the remaining cities by choosing the closest each time
    while unvisited_cities:
        # Find the nearest unvisited city
        next_city = min(unvisited_cities, key=lambda city: problem.path_cost(None, current_city, None, [current_city, city]))
        path.append(next_city)  # Add the closest city to the path
        unvisited_cities.remove(next_city) # Mark the city as visited
        current_city = next_city # Move to the next city

    # Calculate the total cost of the path found
    total_cost = problem.path_cost(None,None, None, path)
    return path, total_cost

# Running the greedy algorithm for the TSP
greedy_route, greedy_cost = greedy_tsp(tsp)
print("Best Route:", greedy_route)
print("Best Cost:", greedy_cost)


In [ ]:
# Visualize the graph with the updated node colors and the path defined by the greedy algorithm
node_colors = {node: 'white' for node in locations.keys()}
node_colors = {node: 'green' for node in locations.keys() if node in greedy_route }

show_map3(romania_graph_data,node_colors,greedy_route)

Given the obtained route shown in the figure above and the computed total distance, we can say that this approach does not seem very effective. Let's explore other algorithms that might yield better results, specifically focusing on how local search methods perform.

<h1 align="left"> <font color='darkblue'> <b>Part III: Local search algorithms </b></font></a></h1>  

In this section, we explore the three local search algorithms covered in class: **Hill Climbing**, **Simulated Annealing**, and the **Genetic Algorithm**. Let's start with the first one, the **Hill Climbing algorithm**.


<h2 align="left"> <b>II.1.Hill Climbing </b> </h2>
**Hill Climbing** is an optimization algorithm used to find a good solution by iteratively moving towards a state that has a higher value based on a given heuristic. The algorithm starts with an initial solution and continually moves to a neighboring state that improves the objective, until no better neighboring states are found. However, it may get stuck in **local optima** because it only considers the immediate best move.


Overall, the algorithm works as follows:
 * Evaluate the initial state.
 * If the initial state meets the goal, return it.
 * Otherwise, find a neighboring state (one that is heuristically similar to the current state)
 * Evaluate this neighboring state. If it is closer to the goal than the current state, replace the current state with this new state and repeat these steps until no improvement is possible.

Below is an example implementation of the Hill Climbing algorithm.

In [10]:
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with better value,
    stopping when no neighbor is better. Returns the final state."""

    # Start from the initial state of the problem
    current = Node(problem.initial)
    while True:
        # Expand the current node to get all neighboring nodes
        neighbors = current.expand(problem) # Look at the adjacent cities

        if not neighbors:  # If there are no neighbors, stop the search
            break

        # Select the neighbor with the better value according to the problem's value function. In this case, we aim to minimize the total distance of the route.
        neighbor = min(neighbors, key=lambda node: problem.value(node.state)) # Choose the best neighbor based on value

        if problem.value(neighbor.state) > problem.value(current.state):
            # If the selected neighbor has a higher value (i.e., worse than the current state), stop
            break

        # Update the current state to the chosen neighbor (since it has a lower value, it is "better")
        current = neighbor
    # Return the final state and its value after the search ends
    return current.state, problem.value(current.state)

In [ ]:
# Using the hill_climbing search on the TSP_problem:
solution, value = hill_climbing(tsp)
print("Hill Climbing solution:", solution)
print("Total distance:", value)

In [ ]:
# Visualize the graph with the updated node colors and the path defined by the HC algorithm
node_colors = {node: 'white' for node in locations.keys()}
node_colors = {node: 'green' for node in locations.keys() if node in solution }

show_map3(romania_graph_data,node_colors,solution)

<font color='darkred'> **How do you justify this result?**



### **Neighbor generation**

There are several ways to generate successors for Hill Climbing.
1. by considering only adjacent cities (as performed in the previous code)
2. by swapping any pair of cities in the path, 
3. by reversing part of the path (e.g., reverse the BCD sequence in [ABCDE] to get [ADCBE] as the successor).


The current method of choosing neighbors is not well-suited for the Traveling Salesperson Problem. We need a neighboring state that yields a total path distance similar to or better than the current state. Therefore, the neighbor generation function needs to be improved.



We suggest testing the third strategy (reversing part of the path) to see if it enhances the effectiveness of Hill Climbing. Note that the function two_opt, defined in the TSP_problem class, implements this strategy.

In [13]:
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with lowest value,
    stopping when no neighbor is better. Returns the final state. """

    # Function to find neighbors using the two_opt method
    def find_neighbors(state, number_of_neighbors=100):
        """Generates neighbors by repeatedly applying the two_opt method."""
        neighbors = []
        # Generate a specified number of neighboring states
        for i in range(number_of_neighbors):
            new_state = problem.two_opt(state)    # Create a new state by applying the two_opt heuristic
            neighbors.append(Node(new_state))     # Store the new state as a Node in the neighbors list
            state = new_state                     # Update the current state to the newly generated state

        return neighbors # Return the list of neighboring nodes

    # Initialize variables
    iterations = 1000  # Total number of iterations for the hill climbing process
    fit_list = []       # List to store the fitness values at each iteration
    states = []         # List to store the states explored during the process
    current = Node(problem.initial)     # Start from the initial state of the problem as a Node
    fit_list=[problem.value(current.state) ]    # Record the fitness value of the initial state

    # Main loop for the hill climbing process
    while iterations:
        neighbors = find_neighbors(current.state)    # Find neighbors of the current state
        
        if not neighbors: # If no neighbors are found, stop the process
            break

        # Select the neighbor with the lowest value (i.e., the best in a minimization context)
        neighbor = min(neighbors, key=lambda node: problem.value(node.state))

        # If the value of the selected neighbor is better (lower) than the current state, move to that neighbor
        if problem.value(neighbor.state) < problem.value(current.state):
            current.state = neighbor.state  # Update the current state to the better neighbor

        iterations -= 1 # Decrease the number of remaining iterations
        states.append(current.state)     # Add the current state to the list of explored states
        fit_list.append(problem.value(current.state))  # Store the value of the fitness function

    # Return the final state, its fitness value, and the history of states and fitness values
    return current.state, problem.value(current.state),states,fit_list

We can now generate an approximate solution to the problem by calling hill_climbing. The results may vary slightly each time it is run.

In [ ]:
# Using the hill_climbing search on the TSP_problem:
solution, value,states,fit_list= hill_climbing(tsp)
print("Hill Climbing solution:", solution)
print("Total distance:", value)

The solution looks like this.

In [ ]:
# Visualize the graph with the path defined by the HC algorithm and two_opt strategy 

node_colors = {node: 'white' for node in locations.keys()}
node_colors = {node: 'green' for node in locations.keys() if node in solution }

show_map3(romania_graph_data,node_colors,solution)

To better understand how the solution evolves over time during the Hill Climbing process, we can plot the fitness values recorded at each iteration (variable _fit_list_). This plot will allow us to visually observe the algorithm's progress and see whether the fitness improves or stagnates. The following code displays a plot of these fitness values:

In [ ]:
plt.plot(fit_list,marker="o")

To evaluate the variability and effectiveness of the Hill Climbing algorithm, we will run it multiple times and record the results. Since hill climbing is often influenced by the initial state and random choices during neighbor selection, running it several times helps us assess the consistency of the solutions.

In the following code, we run the Hill Climbing algorithm 20 times and collect the resulting tour total distance in a list called sim. Afterward, we print the list of tour lengths and identify the shortest tour found across all runs:

In [ ]:
sim=[]
for i in range(20):
    state, value, states,fit_list = hill_climbing(tsp)
    sim.append(value)
print(sim)
print('Best tour length found using Hill Climbing is:', min(sim))

**Remark:** Keep in mind that hill climbing can get stuck in local optima, meaning it may not always find the best possible solution. Depending on the problem instance and the landscape of the solution space, it may be beneficial to run hill climbing multiple times from different starting points or to use other algorithms, such as simulated annealing, which includes a mechanism to escape local optima.

#### <font color='darkred'>**Exercice 1**</front>
<font color='darkred'>Try to implement your own **successor (neighboring) function** for Hill Climbing. Experiment with different approaches.</front>

<font color='darkred'>

- **Objective**: Test how different successor generation strategies can affect the algorithm's ability to escape local optima.
- **Questions**: How does your custom successor function compare to the one provided? Does it improve or worsen the solution quality? Why might this be?
</front>

<h2 align="left"> <b>II.2. Simulated Anealling </b> </h2>

**Applying Simulated Annealing to the Traveling Salesman Problem to find the shortest tour across all cities in Romania.**

**Note:** The intuition behind **Hill Climbing** comes from the idea of climbing up the graph of a function to find its peak. In the case of **maximization**, the algorithm takes one step at a time, always moving towards a higher value, in the hope of reaching the highest peak. However, if we start from the shoulder of a suboptimal hill (e.g., the second-highest peak), the algorithm will converge to this **local maximum** and cannot escape to find the global peak.

For **minimization** problems, such as the Traveling Salesman Problem where we aim to minimize the total distance, the same issue applies in reverse: the algorithm tries to find the lowest point but can get stuck in a **local minimum**, unable to find the best (global) solution. Additionally, Hill Climbing struggles with flat regions of the solution space, where all neighboring states have the same value, making it difficult to make progress towards a global minimum.

Now, let's consider an algorithm that can handle these situations more effectively. **Simulated Annealing** is quite similar to Hill Climbing but introduces randomness to help escape local optima. Instead of always picking the best move, it makes random moves. If the random move brings us closer to the global optimum, it is accepted. If not, the algorithm may still accept it based on a probability influenced by the current **"temperature"**. When the temperature is high, the algorithm is more likely to accept a bad move, allowing greater exploration. As the temperature decreases, it becomes more selective, usually accepting only better moves, with occasional exceptions. This flexibility allows Simulated Annealing to explore the state space thoroughly, reducing the risk of getting stuck in local optima and helping to find a better overall solution.

The temperature is gradually decreased over the course of the iterations using a scheduling routine. 
The most common temperature schedule is **simple exponential decay**:
$$T(t) = \alpha^t T_0$$
where $T_0$ is the initial temperature, $\alpha$ is the decay parameter (with a value typically between 0 and 1), and $t$ is the current iteration.

The implementation presented below uses an exponential decay to gradually reduce the temperature over time.

In [19]:
def exponential_schedule(initial_temperature=1e4, alpha=0.95, iteration=1000):
    """Calculate temperature based on exponential decay.
    
    Args:
        initial_temperature (float): The initial temperature at the start of the algorithm.
        alpha (float): The rate of decay for the temperature, usually a value between 0 and 1.
        iteration (int): The current iteration number.
        
    Returns:
        float: The temperature at the given iteration.
    """
    return  lambda t: (initial_temperature * np.exp(-alpha * t) if t < iteration else 0)

In most cases, the valid range for the initial temperature $T_0$ can be determined experimentally based on the problem; it should generally be set high enough to allow sufficient exploration of the solution space at the beginning. The decay parameter $\alpha$ should be close to, but less than $1.0$ (e.g., $0.90$ to $0.99$). This ensures that the temperature decreases gradually, allowing the system to slowly converge to an optimal solution.

Selecting the right values for $T_0$ and $\alpha$ is crucial, as it involves balancing exploration (broadly searching the solution space) and exploitation (focusing on refining the current solution) to ensure that the simulated annealing process effectively navigates to a near-optimal solution.

#### <font color='darkred'>**Exercice 2**</front>
<font color='darkred'>

Implement **Simulated Annealing** to solve the **Traveling Salesman Problem (TSP)** using an **exponential cooling schedule**.

**Task:**
1. Implement the function `simulated_annealing_tsp(problem, schedule=exponential_schedule(), max_iterations=1000)`.
2. Use the provided `exponential_schedule` function to manage the temperature.
3. Start from an initial route and, at each iteration, select a neighboring route by slightly modifying the current one (here consider the adjacent cities approach).
4. Accept better solutions, or worse solutions with a probability determined by the current temperature.

**Remark:**
- The standard **Simulated Annealing** algorithm is initially defined for **maximization problems**, which is why it uses an acceptance probability of: $P = e^{-\Delta E / T}$, where $\Delta E = \text{current value} - \text{neighbor value}$ represents the change in value, and **value** refers the total distance of the current or neighbor route.

- Since TSP is a **minimization problem**, we need to adapt the **acceptance criteria** as follows:
    - If $\Delta E > 0$, always accept the neighbor since it improves the solution (i.e., lower distance).
    - If $\Delta E < 0$, accept the neighbor with a probability: $P = e^{\Delta E / T}$
</front>

In [20]:
def simulated_annealing(problem, schedule=exponential_schedule(),max_iterations=10000):
    """Performs simulated annealing search.

    Args:
    - problem: An instance of the TSP problem
    - schedule: a function that takes an iteration count and returns a temperature.
    - max_iterations (int): The maximum number of iterations.

    Returns:
    -  The final state, its value, the list of states encountered and their values.
    """
    
    # Your code here



After implementing, run the following

In [ ]:
state, value,states,fit_list = simulated_annealing(tsp)
#print("Simulated annealing solution:", state)
print("SA Total distance:", value)

In [ ]:
# Display the map with updated node colors, highlighting the cities included in the final state
node_colors = {node: 'white' for node in locations.keys()}
node_colors = {node: 'green' for node in locations.keys() if node in state }

show_map3(romania_graph_data,node_colors,state)

In [ ]:
# Plot the fitness values recorded over iterations using a line graph.
plt.plot(fit_list,marker="o")

Running Simulated Annealing multiple times is crucial due to its stochastic nature, helping to assess solution consistency, avoid local optima, and identify the best possible outcome. This increases the reliability of the results by mitigating the effects of randomness.

In [ ]:
sim=[]
for i in range(20):
    state, value,states,fit_list = simulated_annealing(tsp)
    sim.append(value)
print(sim)
print('Best tour length found using SA: ', min(sim))

### <font color='darkred'>**Exercice 3**</front>
<font color='darkred'>

1. Why might **Simulated Annealing** be more effective for solving the TSP compared to **Hill Climbing**?

2. Modify the parameters $T_0$ (initial temperature) and $\alpha$ (decay rate). How do these changes affect convergence, solution quality, and the ability to escape local optima? Reflect on how different configurations influence the overall performance of the algorithm.
    
3. Test **Simulated Annealing** with an alternative temperature schedule, such as **linear**, **logarithmic**, or **quadratic** (try just one of them). Reflect on how changing the temperature schedule influences the behavior and results of the simulated annealing algorithm. Does the modified schedule maintain or improve the efficacy of the algorithm?
   </front>

<h2 align="left"> <b>II.3.  Genetic Algorithm </b></h2>

Genetic algorithms (GA) are inspired by natural evolution and are particularly effective for optimization and search problems involving large state spaces.
n GA, a population of potential solutions (also called individuals or states) is maintained, where each solution represents a feasible candidate. At each iteration (referred to as a generation), the population is evolved using biologically-inspired techniques such as **crossover**, **mutation**, and **natural selection**.

### **Overview**

A genetic algorithm follows these steps:

1.	**Initialize a random population** of potential solutions.
2.	**Calculate the fitness** of each individual in the population.
3.	**Select individuals** based on their fitness to participate in mating.
4.	**Mate the selected individuals** (using crossover and mutation) to create a new generation.
5.	**Repeat steps 2-4** until an individual with a satisfactory fitness is found or a maximum number of generations is reached.


### **Glossary**

Before proceeding, here is a summary of the basic terminology used in genetic algorithms:

* **Individual/State:** A list of elements (referred to as *genes*) that represents a possible solution.
* **Population:** The entire collection of individuals or solutions.
* **Gene Pool:** The set of all possible values that a gene can take, defining the range of potential characteristics for individuals.
* **Generation/Iteration:** One cycle of updating the population, including fitness evaluation, selection, crossover, and mutation.
* **Fitness:** A measure of an individual's quality or suitability, calculated using a problem-specific function to evaluate how "good" the solution is.

#### **1. Initialization**

In the initialization step of a Genetic Algorithm, a population of potential solutions (often called "individuals" or "chromosomes") is generated randomly to provide a diverse starting point for the search process. The following code cell provides an implementation for this step.

In [30]:
def init_population(problem,population_size):
    """Initialize a population of routes (lists of cities)."""
    return [random.sample(problem.initial, len(problem.initial)) for _ in range(population_size)]


#### **2. Fitness function**

Once the initial population is generated, the next step is to evaluate the **fitness** of each individual. Fitness is a measure of how good a solution is in terms of solving the problem at hand. For the Traveling Salesman Problem, the fitness is represented by the total route distance. In this case, we use the method implemented in the _TSP_Problem_ class to compute the path cost.

In [31]:
def fitness(problem,state):
    return problem.path_cost(None, None, None, state) 

#### 2. **Selection**

The **selection phase** in a GA chooses individuals from the current population to serve as parents for the next generation, based on their fitness. This process mimics **natural selection**, favoring individuals with higher fitness while maintaining genetic diversity to prevent premature convergence.

The **selection procedure** can be summarized as follows:

1. Individuals are evaluated using the fitness function.
2. Individuals are chosen based on specific selection strategies, such as **greedy selection**, **roulette wheel selection**, **rank selection**, or **elitism**.

We propose here implementing a combined approach of **Elitism** and **Roulette Wheel Selection**:

- **Elitism**: Directly selects the top individuals (elites) based on their fitness, ensuring that the best solutions from the current generation are preserved in the next generation.

- **Roulette Wheel Selection** (also known as **Fitness Proportionate Selection**): The remaining population is selected based on a probability proportional to their fitness. Individuals with higher fitness have a higher likelihood of being selected, but those with lower fitness still retain a chance. Typically, the probability of selecting an individual $i$ from a population $P $ is calculated as:

  $$P(i) = \dfrac{fitness(i)}{\sum_{k \in P} fitness(k)}$$

In [32]:
def select_elitismRoulette(problem, population, n_population, elite_rate):
    """
    Select individuals from the population using a combination of elitism and roulette wheel selection
    for a minimization problem.

    Args:
        problem (TSP_problem): An instance of the TSP problem to solve.
        population (list): Current population of routes (lists of cities).
        n_population (int): The number of individuals to select for the next generation.
        elite_rate (float): The proportion of the population to retain as elites (0 < elite_rate < 1).

    Returns:
        list: The selected individuals for the next generation.
    """
    # Step 1: Calculate the number of elites
    n_elites = max(1, int(elite_rate * n_population))  # Ensure at least one elite

    # Step 2: Evaluate the cost (Lower distance means better fitness, we use the inverse function, we add 1 to prevent division by zero)
    fitness_scores = [(1 / (1 + fitness(problem,individual)), individual) for individual in population]
    # We use `1 / (1 + cost)` to ensure that lower costs (better solutions) are assigned higher fitness values.
    # Adding 1 avoids division by zero for very small costs.

    # Step 3: Sort population by fitness in descending order (best individuals first)
    fitness_scores.sort(reverse=True)
    sorted_population = [individual for _, individual in fitness_scores]

    # Step 4: Select elites (directly copy top elite individuals)
    elites = sorted_population[:n_elites]

    # Step 5: Perform roulette wheel selection to fill the remaining population slotsvgfrde
    total_fitness = sum(fitness for fitness, _ in fitness_scores)
    selection_probabilities = [fitness / total_fitness for fitness, _ in fitness_scores]

    selected = []
    while len(selected) < (n_population - n_elites):
        # Select one individual based on its probability weight
        selected_individual = random.choices(sorted_population, weights=selection_probabilities, k=1)[0]
        selected.append(selected_individual)

    # Step 6: Combine elites and selected individuals to form the new population
    next_generation = elites + selected

    return next_generation

##### <font color='darkred'>**Exercice 4.1**</front>
 - <font color='darkred'>Implement a selection strategy that combines **elitism** and **rank selection**. In **Rank selection**, individuals are chosen based on theirrelative ranking within the population rather than their absolute fitness scores. </front>
##### <font color='darkred'>**Exercice 4.2**</front>
- <font color='darkred'>Implement the **tournament selection** method. In **tournament selection**, a set number of individuals are randomly selected from the population, and the one with the highest fitness in the group is chosen as the first parent. This process is repeated to select the second parent.  This process is repeated to select the second parent. Tournament selection helps ensure that fitter individuals are more likely to be chosen while still allowing some opportunity for weaker individuals, adding diversity.</front>

#### **3. Crossover**

In genetic algorithms, two individuals can "mate" to produce an offspring. This offspring inherits characteristics from both parents, combining their strengths to form a potentially better solution. There are several ways to implement crossover, and most variations are derived from the common methods described below:

* **Point Crossover**: In **point crossover**, one or more points are selected along the parent individuals, where they will be "split." The segments from each parent are then combined to form an offspring. In the example below, two parents are split at the 3rd position, and segments are exchanged, resulting in an offspring that inherits traits from both parents.

  ![point crossover](images/point_crossover.png)

* **Uniform Crossover**: In **uniform crossover**, each gene is chosen randomly from either parent, rather than splitting at fixed points. In the example below, genes 1, 4, and 5 were selected from the first parent, while genes 2 and 3 were taken from the second parent. This method allows for a more even mix of genetic material from both parents.

  ![uniform crossover](images/uniform_crossover.png)

Here is an implementation for one point crossover.

In [35]:
def one_point_crossover(parent1, parent2):

    # Choose one crossover point
    crossover_point = random.randint(0, len(parent1)-1)

    # Start by copying genes from parent1 up to the crossover point
    child = parent1[:crossover_point]

    # Then fill the rest of the child with genes from parent2 that aren't already in the child
    for city in parent2:
        if city not in child:
            child.append(city)

    return child

##### <font color='darkred'>**Exercice 4.3**</front>
- <font color='darkred'>Propose an implementation for **two-points crossover**.</front>
##### <font color='darkred'>**Exercice 4.4**</front>
- <font color='darkred'>Propose an implementation for **uniform crossover**.</front>

#### 4. **Mutation**

When an offspring is produced, there is a chance it will **mutate**, meaning one or more of its genes may be altered.

For example, consider an individual represented by the sequence `"abcde"`. If it is selected for mutation, we might randomly choose to change its third gene to `'z'`, resulting in the new sequence `"abzde"`. This mutated individual is then added to the population, contributing new genetic diversity.

Setting the **mutation rate** in a GA is crucial for balancing **exploration** and **exploitation** during the search process. The **mutation rate** determines the likelihood that a gene in an individual will undergo mutation.

- **High Mutation Rate**:
  - Increases diversity within the population and helps prevent premature convergence to local optima.
  - However, it may also disrupt well-adapted solutions, making the search process behave more like a random walk, which can slow down convergence.

- **Low Mutation Rate**:
  - Preserves high-quality solutions and allows for more steady, incremental improvements.
  - The downside is the risk of getting stuck in local optima, as there may not be enough diversity to explore other areas of the solution space.

**How to Set and Adjust the Mutation Rate**:

- **Relation to Population Size**: If the population size is large, a lower mutation rate may be sufficient since the number of individuals provides enough diversity. Conversely, for a smaller population, a higher mutation rate might be needed to maintain variability.

- **Iterative Adjustment**: Similar to other hyperparameters in optimization algorithms, setting the mutation rate often requires an iterative approach—testing, analyzing results, adjusting, and retesting. This allows you to find the optimal balance between exploration and exploitation for your specific problem.

In [38]:
def mutate(route,mutation_rate):
    # with permutation encoding, two numbers (cities) are selected and exchanged
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]
    return route


#### 5. **Genetic algorithm**
Now that we have defined the various intrinsic components of the Genetic Algorithm, the goal is to bring everything together to construct the complete algorithm.

The algorithm takes the following inputs:

* **`problem`**: Represents the problem that the genetic algorithm aims to solve.

* **`population_size`**: Specifies the number of individuals in each generation's population.

* **`generations`**: Defines the number of iterations (generations) the genetic algorithm will run.

* **`mutation_rate`**: Represents the probability of mutation. In typical genetic algorithms, mutation rates range from **0.001** to **0.1**. However, given the smaller population size in this context, a higher mutation rate may be beneficial. A mutation rate between **0.05** to **0.1** (indicating a 5% to 10% chance of mutation per gene) can be an effective starting point.

* **`elite_rate`**: Represents the proportion of the top-performing individuals (elites) that should be directly passed on to the next generation without undergoing crossover or mutation. Typical values for elite_rate are between **0.05** and **0.2**, ensuring the preservation of high-quality solutions.

The algorithm outputs the **best state/route** with the highest fitness score.

##### <font color='darkred'>**Exercice 5**

- Combine the functions you have implemented in previous exercises to build the complete **Genetic Algorithm** for solving the **Traveling Salesman Problem**.
- Make sure to include all components: **initial population generation**, **fitness evaluation**, **selection**, **crossover**, **mutation**, and **replacement**.
- For **selection**, use a combined **elitism and roulette wheel selection** strategy.
- For **crossover**, use a **two-point crossover** technique to generate offspring.
- For **mutation**, use a **permutation-based mutation** method such as **two-opt**, which randomly reverses segments of the route to introduce variability.
- Test your complete implementation to find the shortest possible route.
- Document your code to explain how each component contributes to solving the problem.</front>

In [ ]:
def genetic_algorithm(problem, generations=1000, population_size=100, mutation_rate=0.1, elite_rate=0.1):
    """
    Genetic Algorithm for solving the Traveling Salesman Problem (TSP).

    Args:
        problem (TSP_problem): An instance of the TSP problem to solve.
        generations (int): The number of generations to run.
        population_size (int): The size of the population for each generation.
        mutation_rate (float): The probability of mutation for each individual.
        elite_rate (float): The proportion of elites to keep in each generation (0 < elite_rate < 1).

    Returns:
        tuple: The best state (route), its corresponding cost, and a list of best costs for each generation.
    """
    # Your code here

    return best_individual, best_cost, best_cost_evolution

# Example usage:
# Assuming `tsp` is an instance of the TSP_problem class with a given set of cities and distance matrix
best_route, best_cost, best_cost_evolution = genetic_algorithm(tsp)

print("Best Route:", best_route)
print("Shortest Distance:", best_cost)



In [ ]:
node_colors = {node: 'white' for node in locations.keys()}
node_colors = {node: 'green' for node in locations.keys() if node in best_route }

show_map3(romania_graph_data,node_colors,best_route)

In [ ]:
plt.plot(best_cost_evolution,marker="o")

In [ ]:
sim=[]
for i in range(20):
    best_route, best_cost, fit_list = genetic_algorithm(tsp)
    sim.append(best_cost)
print(sim)
print('Best tour distance found using GA: ', min(sim))

##### <font color='darkred'>**Exercise 6**

Update the **Genetic Algorithm** function with the following configurations:

1. **Employ elitism-rank for population selection and a uniform crossover mechanism**.
2. **Adopt tournament selection and a two-point crossover technique**.

* **Discussion**: Analyze how these two configurations impact the performance of the genetic algorithm. Consider metrics such as convergence speed, diversity of solutions, and the quality of the final solution.

</front>